In [10]:
import sctk

In [29]:
import anndata as ad

# starting from the data after clean_sc_counts
data_dir = "/Users/arturszalata/PycharmProjects/task-dge-perturbation-prediction/resources/neurips-2023-data"

In [57]:
sc_counts = ad.read_h5ad(f'{data_dir}/sc_counts_cleaned.h5ad')

In [35]:
sctk.calculate_qc(sc_counts, flags={"mito": r"^MT-", "ribo": r"^RP[LS]"})

In [36]:
sctk.cellwise_qc(sc_counts)

n_counts: [2358.802254718472, 29992.009765625], 299272/301785 passed
n_genes: [861.7906960906522, 6238.99853515625], 301402/301785 passed
percent_mito: [0.025329280644655228, 19.193623063602075], 301119/301785 passed
percent_ribo: [6.9978885607387955, 50.10729574034552], 295113/301785 passed
291883/301785 pass


In [43]:
sc_counts = sc_counts[sc_counts.obs.cell_passed_qc]

In [51]:
sc_counts.obs['cell_type'] = sc_counts.obs['cell_type'].apply(lambda x: 'T cells' if 'T cells' in x else x)

/var/folders/98/9ygjwkpd12d1vxydfprn1st00000gp/T/ipykernel_80691/4188281325.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  sc_counts.obs['cell_type'] = sc_counts.obs['cell_type'].apply(lambda x: 'T cells' if 'T cells' in x else x)


In [149]:
sc_counts.obs["library_well"] = sc_counts.obs["library_id"].astype(str) + "_" + sc_counts.obs["well"].astype(str)
# remove samples that belong to library_well groups with less than 200 cells
sc_counts = sc_counts[sc_counts.obs["library_well"].map(sc_counts.obs["library_well"].value_counts()) >= 200]

In [ ]:
# filter out genes expressed in less than 100 cells per plate

In [162]:
import numpy as np
import pandas as pd

plates = sc_counts.obs['plate_name'].unique()

# Step 2: Initialize a DataFrame to store counts
gene_counts_per_plate = pd.DataFrame(index=sc_counts.var_names, columns=plates, dtype=int)

# Step 3: Iterate over each plate and calculate expression counts
for plate in plates:
    # Subset the AnnData object for the current plate
    subset = sc_counts[sc_counts.obs['plate_name'] == plate]
    
    # Calculate expression counts (genes x cells > 0)
    expressed_genes = (subset.X > 0).sum(axis=0)
    
    # Check if the result needs conversion from sparse matrix format
    if isinstance(expressed_genes, np.matrix):
        expressed_genes = np.array(expressed_genes).flatten()
    
    # Store the counts in the DataFrame
    gene_counts_per_plate[plate] = expressed_genes

# Step 4: Aggregate counts across plates (max or sum based on the requirement)
# We use `max` here to find if any gene meets the criteria in at least one plate
max_counts = gene_counts_per_plate.max(axis=1)

# Step 5: Create a mask for genes to keep (genes expressed in at least 100 cells in any plate)
genes_to_keep = max_counts >= 100

# Filter the AnnData object to keep only these genes
sc_counts = sc_counts[:, genes_to_keep]

In [166]:
del sc_counts.uns

In [167]:
sc_counts.write_h5ad("../data/sc_counts_filtered.h5ad")